# Code 4 - SQL Agents with Dynamic Few Shot Prompting

## Step 1 - Connect to Database

### Import Packages

In [1]:
from sqlite3 import connect
from langchain_community.utilities.sql_database import SQLDatabase

### Create database connection for LLM (create db)

In [2]:
db = SQLDatabase.from_uri("sqlite:///chinook.db", sample_rows_in_table_info = 3)
print(len(db.get_usable_table_names()), db.get_usable_table_names())

11 ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


### Check Connection

In [3]:
con = connect("chinook.db")
cur = con.execute("Select Count(Distinct(AlbumId)) from Album;")
print(cur.fetchall())

[(347,)]


## Step 2 - Setup Langchain SQL Chain

### Import API Key

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
API_KEY = os.getenv("OPENAI_API_KEY")

### Import Packages

In [6]:
from langchain.callbacks import get_openai_callback
from langchain_groq import ChatGroq

### Create an Instance of LLM with configuration & Check (creating llm)

In [7]:
llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017B84FDD010>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017B84FDEB50>, temperature=1e-08, groq_api_key=SecretStr('**********'))

## Step 3 Setup Agent with Few Shot Examples

Advantages of SQL Agents:

1. Saves tokens - Retrieving only required schema & relevant tables.
2. Recover from errors by generating queries, catch the traceback and regenerate correctly
3. Chains run on a predefined pattern; Agents is defined by llm; The LLM then chooses the right tools to execute

### Import Embedding Packages

In [27]:
from langchain_openai import OpenAIEmbeddings

### Create an Instance of Embeddings

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=os.environ.get("OPENAI_API_KEY"))

### Sample Embedding

In [10]:
#len(embeddings.embed_query("Hi"))

### Create few Shot Examples

In [11]:
examples = [
    {   "input": "List all artists.", 
        "query": "SELECT * FROM Artist;"},
    {
        "input": "Find all albums for the artist 'AC/DC'.",
        "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
    },
    {
        "input": "List all tracks in the 'Rock' genre.",
        "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
    },
    {
        "input": "Find the total duration of all tracks.",
        "query": "SELECT SUM(Milliseconds) FROM Track;",
    },
    {
        "input": "List all customers from Canada.",
        "query": "SELECT * FROM Customer WHERE Country = 'Canada';",
    },
    {
        "input": "How many tracks are there in the album with ID 5?",
        "query": "SELECT COUNT(*) FROM Track WHERE AlbumId = 5;",
    },
    {
        "input": "Find the total number of Albums.",
        "query": "SELECT COUNT(DISTINT(AlbumId)) FROM Invoice;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "Employee"',
    },
]
print(len(examples))

10


### Import Vector Store Packages

In [16]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

### Embed Examples

In [17]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=3,
    input_keys=["input"],
    )

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Check Similarity of Query in Vector DB

In [ ]:
example_selector.vectorstore.search("How many arists are there?", search_type = "similarity")

### Import Template Packages

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

### Create System prefix Prompt

In [14]:
system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

In [15]:
dynamic_few_shot_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix=""
)

NameError: name 'FewShotPromptTemplate' is not defined

### Integrate few Shot Prompts with User Inputs

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate

In [19]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=dynamic_few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

#### Check Prompt

In [21]:
print(full_prompt.messages[0].to_json()['kwargs'])

{'prompt': FewShotPromptTemplate(input_variables=['dialect', 'top_k'], examples=[{'input': 'List all artists.', 'query': 'SELECT * FROM Artist;'}, {'input': "Find all albums for the artist 'AC/DC'.", 'query': "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');"}, {'input': "List all tracks in the 'Rock' genre.", 'query': "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');"}, {'input': 'Find the total duration of all tracks.', 'query': 'SELECT SUM(Milliseconds) FROM Track;'}, {'input': 'List all customers from Canada.', 'query': "SELECT * FROM Customer WHERE Country = 'Canada';"}, {'input': 'How many tracks are there in the album with ID 5?', 'query': 'SELECT COUNT(*) FROM Track WHERE AlbumId = 5;'}, {'input': 'Find the total number of Albums.', 'query': 'SELECT COUNT(DISTINT(AlbumId)) FROM Invoice;'}, {'input': 'List all tracks that are longer than 5 minutes.', 'query': 'SELECT * FROM Track WHERE Milliseconds > 300

### Validate Prompt

In [22]:
prompt_val = full_prompt.invoke(
    {
        "input": "How many arists are there?",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't 

In [23]:
dynamic_few_shot_agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
    stream_runnable = False
)

### Execute LLM Agent

In [25]:
with get_openai_callback() as cb:
    out = dynamic_few_shot_agent.invoke({"input": "How many artists are there?"})
print(out)
print(cb)



> Entering new SQL Agent Executor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Check Execution Flow

In [27]:
print(agent_with_few_shots)

name='SQL Agent Executor' verbose=True agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'top_k': '10', 'dialect': 'sqlite'}, messages=[SystemMessagePromptTemplate(prompt=FewShotPromptTemplate(input_variables=['dialect', 'top_k'], examples=[{'input': 'List all artists.', 'query': 'SELECT * FROM Artist;'}, {'input': "Find all albums for the artist 'AC/DC'.", 'query': "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');"}, {'input':

# END - Next Topic Custom Agent